# Trabajo Integrador Técnicas de procesamiento de imágenes

## IFTS 24

Profesor: Matías Barreto

Alumna Gabriela Pari Vaca



---


## Asistente por voz

Descripción del proyecto

El proyecto consiste en una interfaz que permite al usuario interactuar con imagen y voz. Mediante la camara web se podrá sacar una foto y hacerle una pregunta por voz, la respuesta también será mediante la voz.

Tecnologías usadas
- PaliGemma
- Gradio
- Torch
- Transformers de Hugging Face
- gTTS(Google Text-to-Speech)


## Instalación e importación de librerías

In [ ]:
!pip install gradio
!pip install torch transformers
!pip install Pillow
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import gradio as gr
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, pipeline
from PIL import Image
from gtts import gTTS
import os

## Token
Para el uso de la libreria de Paligemma que se encuentra en HuggingFace.

Se necesita tener creado un usuario con las credenciales. https://huggingface.co/settings/tokens

Para el uso de este notebook se puede ir a la sección de "secrets" crear una clave con el nombre de 'HF_TOKEN'. Otra opción es usar el código de "notebook_login" donde se copia el token sin guardarlo

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

#notebook_login(hf_token)
login(hf_token)

## 1. Configuración

- Cargar el modelo VLM(visual-language-model) de Paligemma 3b
- Configurar la máquina para que use GPU
- Speech-to-Text **openai/whisper-base**

In [ ]:
print("Cargando modelo PaliGemma")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
model_id = "google/paligemma-3b-mix-224"
processor = AutoProcessor.from_pretrained(model_id)
pali_model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=model_dtype,
    device_map=device,
    revision="bfloat16",
).eval()

# --- Pipeline de Speech-to-Text (Whisper) ---
print("Cargando modelo Whisper (Speech-to-Text)...")
# Usamos un modelo más pequeño para un inicio más rápido.
asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

print("Modelos cargados")


Cargando modelo PaliGemma


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda


Modelos cargados


## 2. Función Principal

In [ ]:
def procesar_foto_y_voz(pil_image, audio_filepath):
    """
    Orquesta todo el proceso:
    1. Transcribe la voz a texto.
    2. Hace la pregunta sobre la imagen al modelo PaliGemma.
    3. Convierte la respuesta de texto a voz.

    """
    # Validar entradas
    if pil_image is None:
        error_msg = "ERROR: Por favor, toma una foto primero."
        gTTS(text=error_msg, lang='es').save("error.mp3")
        return error_msg, "error.mp3"

    if audio_filepath is None:
        error_msg = "ERROR: Por favor, graba una pregunta."
        gTTS(text=error_msg, lang='es').save("error.mp3")
        return error_msg, "error.mp3"

    try:
        # Paso 1: Transcribir la voz a texto
        print(f"Transcribiendo audio desde: {audio_filepath}")
        transcripcion = asr_pipeline(audio_filepath)
        pregunta_texto = transcripcion['text']
        print(f"Texto transcrito: '{pregunta_texto}'")

        if not pregunta_texto:
            error_msg = "No pude entender la pregunta. Por favor, habla más claro."
            gTTS(text=error_msg, lang='es').save("error.mp3")
            return error_msg, "error.mp3"

        # Paso 2: Usar PaliGemma para responder la pregunta sobre la imagen
        prompt = pregunta_texto
        inputs = processor(text=prompt, images=pil_image, return_tensors="pt").to(pali_model.device)
        reply = pali_model.generate(**inputs, max_new_tokens=100)
        respuesta_texto = processor.decode(reply[0], skip_special_tokens=True)
        respuesta_limpia = respuesta_texto.split(prompt)[-1].strip()
        print(f"Respuesta del modelo: '{respuesta_limpia}'")

        # Paso 3: Convertir la respuesta de texto a voz
        output_audio_path = "respuesta.mp3"
        tts = gTTS(text=respuesta_limpia, lang='es', slow=False)
        tts.save(output_audio_path)
        print(f"Audio de respuesta guardado en: {output_audio_path}")

        # Devolver la respuesta en texto y la ruta al archivo de audio de respuesta
        return respuesta_limpia, output_audio_path

    except Exception as e:
        print(f"Ocurrió un error general: {e}")
        error_msg = f"Lo siento, ocurrió un error: {e}"
        gTTS(text="Ocurrió un error inesperado", lang='es').save("error.mp3")
        return error_msg, "error.mp3"


## 3. Creación de la Interfaz



In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="Asistente Visual por Voz") as demo:
    gr.Markdown(
        """
        # Asistente Visual por Voz
        Usa tu cámara para tomar una foto y luego graba una pregunta en español sobre ella.
        """
    )

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Paso 1: Toma una foto", sources=["webcam"], type="pil", elem_id="camera-container")
            audio_input = gr.Audio(label="Paso 2: Graba tu pregunta", sources=["microphone"], type="filepath", elem_id="voice-recorder-container")
            submit_button = gr.Button("Procesar", variant="primary")

        with gr.Column():
            # Salidas
            output_text = gr.Textbox(label="Respuesta en Texto", lines=8, interactive=False)
            output_audio = gr.Audio(label="Respuesta en Audio", type="filepath", autoplay=True)

    # Conectar el botón a la función principal
    submit_button.click(
        fn=procesar_foto_y_voz,
        inputs=[image_input, audio_input],
        outputs=[output_text, output_audio]
    )

if __name__ == "__main__":
    demo.launch() # debug=True te ayuda a ver errores en la consola


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ff51bfcf33b2943db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Próximos pasos a seguir

Mi idea de esta aplicación es aumentar la accesibilidad para los usuarios que tienen dificutad visual. El módulo JS se debe agregar con el parametro de js en gradio.



In [ ]:
# Modulo JS para agregar mayor accecibilidad
js = """
// Función para simular un clic en un botón dentro de un contenedor específico
console.log("se ingreso");
function clickButtonInContainer(containerId) {
    // Buscamos el contenedor por el ID que le dimos en Python (ej: #camera-container)
    const container = document.querySelector(containerId);
    console.log("clic container");
    if (container) {
        // Buscamos el PRIMER botón que se encuentre DENTRO de ese contenedor
        const button = container.querySelector("button");
        if (button) {
            // Si encontramos el botón, le hacemos clic
            button.click();
            console.log(`Botón clickeado dentro de: ${containerId}`);
        } else {
            console.log(`No se encontró un botón dentro de: ${containerId}`);
        }
    } else {
        console.log(`No se encontró el contenedor: ${containerId}`);
    }
}

// Agregamos un "escuchador" de eventos para las pulsaciones de teclas en toda la página
document.addEventListener("keydown", (event) => {
    console.log("se ingreso addeventListener");

    // Si se presiona la tecla "f"
    if (event.key.toLowerCase() === "f") {
        // Prevenimos la acción por defecto del navegador (como abrir la búsqueda)
        event.preventDefault();
        // Llamamos a nuestra función para hacer clic en el botón de la cámara
        clickButtonInContainer("#camera-container");
    }

    // Si se presiona la tecla "v"
    if (event.key.toLowerCase() === "v") {
        event.preventDefault();
        // Llamamos a nuestra función para hacer clic en el botón de grabar voz
        clickButtonInContainer("#voice-recorder-container");
    }
});
"""

## Links

https://huggingface.co/google/paligemma-3b-mix-224